In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is 

In [ ]:
import transformers
import datasets
import accelerate

print(transformers.__version__)
print(datasets.__version__)
print(accelerate.__version__)

4.51.3
3.5.0
1.5.2


Get training data

In [ ]:
import pandas as pd
import os

In [ ]:
work_arrangement_dev = pd.read_csv('data/work_arrangements_development_set.csv')

work_arrangement_dev

,id,job_ad,y_true
0,79484040,Job title: CEO\nAbstract: Exciting opportunity...,Remote
1,80331384,Job title: Home-Based Online ESL Teacher (Onli...,Remote
2,79721069,"Job title: Safeguarding, De La Salle\nAbstract...",Hybrid
3,80190376,Job title: Delivery Driver\nAbstract: Pickup t...,OnSite
4,80082230,Job title: Store Supervisor\nAbstract: We are ...,OnSite
...,...,...,...
94,80151196,Job title: Senior Pipeline Technical Director\...,Hybrid
95,79652545,Job title: Customer Support Administrator\nAbs...,OnSite
96,79718230,Job title: Remote Writing Evaluator for AI (As...,Remote
97,80226188,Job title: People & Culture Advisor\nAbstract:...,Hybrid


In [ ]:
# create fine tuning dataset using dev data
from datasets import Dataset

inputs = []
targets = []

for i in range(len(work_arrangement_dev)):
    # Access job_ad and y_true from the row Series
    job_ad_info = work_arrangement_dev.iloc[i].job_ad
    y_label = work_arrangement_dev.iloc[i].y_true

    prompt = f"Classify the work arrangement of the following job ad as one of the following: onsite, remote, or hybrid: {job_ad_info}"

    inputs.append(prompt)
    targets.append(y_label.lower())


# Create a DataFrame
df = pd.DataFrame({
    "input": inputs,
    "target": targets
})

# Convert to Hugging Face dataset and split
dataset = Dataset.from_pandas(df)

# To do: Remove split due to small data set
dataset = dataset.train_test_split(test_size=0.2)


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import Trainer, TrainingArguments

# Load model and tokenizer
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

In [ ]:
# Tokenize function for your dataset
def preprocess(examples):
    model_inputs = tokenizer(examples["input"], padding="max_length", truncation=True, max_length=512)

    # Tokenize targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["target"], padding="max_length", truncation=True, max_length=8)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply tokenization to both splits
tokenized_datasets = dataset.map(preprocess, batched=True)

Map:   0%|          | 0/79 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [ ]:
import os
os.environ["WANDB_MODE"] = "offline"

In [ ]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    report_to=None,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)

# Train the model
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: a-moramganti (a-moramganti-unsw) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,No log,1.773150
2,No log,0.582265
3,No log,0.193274
4,No log,0.153280


Epoch,Training Loss,Validation Loss
1,No log,1.773150
2,No log,0.582265
3,No log,0.193274
4,No log,0.153280
5,No log,0.153966


TrainOutput(global_step=25, training_loss=2.8189923095703127, metrics={'train_runtime': 893.3748, 'train_samples_per_second': 0.442, 'train_steps_per_second': 0.028, 'total_flos': 53460011581440.0, 'train_loss': 2.8189923095703127, 'epoch': 5.0})

In [ ]:
import pandas as pd
import numpy as np
from transformers import T5Tokenizer, T5ForConditionalGeneration
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

model_path = "./results/checkpoint-25"  # Or wherever the model is saved

# load the original tokenizer and your fine-tuned model
tokenizer = T5Tokenizer.from_pretrained("t5-small")  # Original tokenizer
model = T5ForConditionalGeneration.from_pretrained(model_path)

# load your test data
test_data = pd.read_csv("data/work_arrangements_test_set.csv")  # Adjust path as needed

# Prepare inputs
test_inputs = []
for job_ad in test_data['job_ad']:
    prompt = f"Classify the work arrangement of the following job ad as one of the following: onsite, remote, or hybrid: {job_ad}"
    test_inputs.append(prompt)

# Get labels and convert to lowercase for consistency
true_labels = [label.lower() for label in test_data['y_true']]

# Make predictions
predictions = []
model.eval()  # Set model to evaluation mode

for input_text in test_inputs:
    # tokenize input
    input_ids = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=512).input_ids

    # generate prediction
    outputs = model.generate(input_ids, max_length=8)

    # decode prediction
    pred = tokenizer.decode(outputs[0], skip_special_tokens=True).lower()
    predictions.append(pred)

# calculate metrics
accuracy = accuracy_score(true_labels, predictions)
report = classification_report(true_labels, predictions)
conf_matrix = confusion_matrix(true_labels, predictions)

# print results
print(f"Accuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(report)
print("\nConfusion Matrix:")
print(conf_matrix)

# create a DataFrame with the results for error analysis
results_df = pd.DataFrame({
    'job_ad': test_data['job_ad'],
    'true_label': true_labels,
    'predicted': predictions,
    'correct': [pred == true for pred, true in zip(predictions, true_labels)]
})

# save results
results_df.to_csv("model_evaluation_results.csv", index=False)

# print examples of incorrect predictions for error analysis
print("\nSample of incorrect predictions:")
incorrect = results_df[results_df['correct'] == False].head(5)
for _, row in incorrect.iterrows():
    print(f"Job Ad: {row['job_ad'][:100]}...")
    print(f"True: {row['true_label']}, Predicted: {row['predicted']}")
    print("-" * 50)

Accuracy: 0.4242

Classification Report:
              precision    recall  f1-score   support

      hybrid       0.14      0.04      0.06        27
      onsite       0.63      0.70      0.66        46
      remote       0.22      0.35      0.27        26

    accuracy                           0.42        99
   macro avg       0.33      0.36      0.33        99
weighted avg       0.39      0.42      0.39        99


Confusion Matrix:
[[ 1  7 19]
 [ 1 32 13]
 [ 5 12  9]]

Sample of incorrect predictions:
Job Ad: Job title: Occupational Therapist
Abstract: Paediatric Occupational Therapist
Employer: LaunchPad Th...
True: hybrid, Predicted: remote
--------------------------------------------------
Job Ad: Job title: Client Success Coach
Abstract: Be a key part supporting growth by coaching clients throug...
True: hybrid, Predicted: onsite
--------------------------------------------------
Job Ad: Job title: Software Support /Customer Support Consultant
Abstract: Are you the 'go-to' per

-----------------------------------------
